In [21]:
import numpy as np
import random
import heapq
import itertools

## SWAP Test generator

In [2]:
def swap_test(psi_state, phi_state):
    n = len(psi_state)
    qc = QuantumCircuit(n*2 + 1, 1)
    qc.append(Initialize(psi_state), range(n))
    qc.append(Initialize(phi_state), list(np.arange(n,n*2)))
    qc.h(n*2)

    for qubit in range (n):
        qc.cswap(n*2, qubit, n+qubit)
    
    qc.h(n*2)
    qc.measure(n*2, 0)
    return qc

### Generate all 4x4 image combinations

In [39]:
four_pixel_array = np.array([
    [1,1,1,1]
])
three_pixel_array = np.array([
    [1,1,1,0],
    [1,1,0,1],
    [1,0,1,1],
    [0,1,1,1]
])
two_pixel_array = np.array([
    [1,1,0,0],
    [1,0,1,0],
    [1,0,0,1],
    [0,1,1,0],
    [0,1,0,1],
    [0,0,1,1]
])
one_pixel_array = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1],
])
zero_pixel_array = np.array([
    [0,0,0,0]
])

first_row = (three_pixel_array,four_pixel_array)
second_row = (two_pixel_array,three_pixel_array)
third_row = (one_pixel_array,two_pixel_array)
fourth_row = (zero_pixel_array,one_pixel_array)

ceiling_images = []
floor_images = []

for first_array_list in first_row:
    for second_array_list in second_row:
        for third_array_list in third_row:
            for fourth_array_list in fourth_row:
                for first_array in first_array_list:
                    for second_array in second_array_list:
                        for third_array in third_array_list:
                            for fourth_array in fourth_array_list:
                                data = np.array([
                                    first_array, second_array, third_array, fourth_array
                                ])
                                ceiling_images.append(data)
                                
for first_array_list in first_row:
    for second_array_list in second_row:
        for third_array_list in third_row:
            for fourth_array_list in fourth_row:
                for first_array in first_array_list:
                    for second_array in second_array_list:
                        for third_array in third_array_list:
                            for fourth_array in fourth_array_list:
                                data = np.array([
                                    fourth_array, third_array, second_array, first_array
                                ])
                                floor_images.append(data)

for i in range(len(ceiling_images)):
    ceiling_images[i] = (ceiling_images[i], "ceiling")

for i in range(len(floor_images)):
    floor_images[i] = (floor_images[i], "floor")
    
random.shuffle(ceiling_images)
random.shuffle(floor_images)

ceiling_train = ceiling_images[:2000]
ceiling_test = ceiling_images[2000:]

floor_train = floor_images[:2000]
floor_test = floor_images[2000:]

training_images = ceiling_train + floor_train
test_images = ceiling_test + floor_test

random.shuffle(training_images)
random.shuffle(test_images)

In [41]:
def create_unitary(v):
    dim = v.size
    # Return identity if v is a multiple of e1
    if v[0][0] and not np.any(v[0][1:]):
        return np.identity(dim)
    e1 = np.zeros(dim)
    e1[0] = 1
    w = v/np.linalg.norm(v) - e1
    return np.identity(dim) - 2*((np.dot(w.T, w))/(np.dot(w, w.T)))

In [53]:
ceiling_images[0][0]

array([[1, 0, 1, 1],
       [1, 1, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0]])

In [71]:
vectors_to_sum = []
for i in range (4):
    for j in range(4):
        if ceiling_images[0][0][i][j] == 1:
            print(f"i: {i}, j:{j}")
            state_vector_index = double_index_to_index_mapping[str(i)+str(j)]
            state_vector = pixel_index_to_statevector_mapping[state_vector_index]
            vectors_to_sum.append(state_vector)
vec = np.array([sum(vectors_to_sum)])
create_unitary(vec).round()

i: 0, j:0
i: 0, j:2
i: 0, j:3
i: 1, j:0
i: 1, j:1
i: 1, j:2
i: 2, j:1


array([[-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.],
       [ 0.,  0., -0.,  1.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.],
       [ 0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0

In [58]:
pixel_index_to_statevector_mapping = {
    0:  np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    1:  np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    2:  np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    3:  np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    4:  np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    5:  np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    6:  np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    7:  np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
    8:  np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
    9:  np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
    10: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
    11: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
    12: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
    13: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
    14: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
    15: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
}

double_index_to_index_mapping = {
    "00": 3,
    "01": 7,
    "02": 11,
    "03": 15,
    "10": 2,
    "11": 6,
    "12": 10,
    "13": 14,
    "20": 1,
    "21": 5,
    "22": 9,
    "23": 13,
    "30": 0,
    "31": 4,
    "32": 8,
    "33": 12,
}

In [72]:
def perform_unitary_check(W):
    #array_to_latex(W, pretext="\\text{W} = ")
    W_transposed = np.transpose(W)
    #array_to_latex(W_transposed, pretext="\\text{W}^* = ")
    result = np.dot(W, W_transposed).round()
    #array_to_latex(result, pretext="W^*W = ")
    return np.all(np.equal(result, np.eye(result.shape[0])))


True